# `Nuqleon.Json.Serialization`

Provides fast JSON serialization for objects using runtime code compilation for specialized parsers and printers.

> **Note:** The functionality in this assembly has been used to provide high throughput serialization and deserialization using JSON, for objects of a given static type. It allows for efficient skipping of tokens that are irrevelant to the object being deserialized, beating performance of `Newtonsoft.Json` hands down. Alternatives with `System.Text.Json` have not been explored; this assembly predates the inclusion of JSON support in .NET by several years.

## Reference the library

### Option 1 - Use a local build

If you have built the library locally, run the following cell to load the latest build.

In [1]:
#r "bin/Debug/net50/Nuqleon.Json.Serialization.dll"

### Option 2 - Use NuGet packages

If you want to use the latest published package from NuGet, run the following cell.

In [1]:
#r "nuget:Nuqleon.Json.Serialization,*-*"

## (Optional) Attach a debugger

If you'd like to step through the source code of the library while running samples, run the following cell, and follow instructions to start a debugger (e.g. Visual Studio). Navigate to the source code of the library to set breakpoints.

In [1]:
System.Diagnostics.Debugger.Launch();

## Strongly typed JSON serialization

The serializer provided by this library is parameterized on a data type which is used to derive efficient parsers and printers at runtime. To illustrate this capability, we'll first define a custom type.

In [1]:
record Person
{
    public string Name { get; init; }
    public int Age { get; init; }
}

### Serialize

To serialize objects to JSON, we first create an `IFastJsonSerializer<T>` by using the `FastJsonSerializerFactory.CreateSerializer<T>(INameProvider, FastJsonSerializerSettings)` method. The two parameters provided are:

* An `INameProvider` to convert property and field names to the names used in the corresponding JSON objects.
* Settings represented as a `FastJsonSerializerSettings` object. This currently only consist of a concurrency mode, but may be extended in the future.

To illustrate serialization, let's have a look at creating a serializer first.

In [1]:
using Nuqleon.Json.Serialization;

IFastJsonSerializer<Person> serializer = FastJsonSerializerFactory.CreateSerializer<Person>(DefaultNameProvider.Instance, new FastJsonSerializerSettings(FastJsonConcurrencyMode.SingleThreaded));

In this configuration, we use the default name provider which will use the `Name` and `Age` property names when emitting a JSON object for a `Person` instance. The serializer settings specify a `SingleThreaded` concurrency mode, which makes the serializer only safe to run from a single thread. This enables higher throughput compared to a concurrency-safe instance of a serializer.

Next, we can use the serializer to serialize a `Person` instance.

In [1]:
var p = new Person { Name = "Bart", Age = 21 };

string json = serializer.Serialize(p);

Console.WriteLine(json);

{"Age":21,"Name":"Bart"}


Another overload of `Serialize` accepts a `TextWriter` to append the JSON string to, thus avoiding the allocation of an intermediate string.

At a first glance, this doesn't look much different from other serialization frameworks. However, by passing the `Person` type to `CreateSerializer<T>`, the library can generate optimized code to perform serialization. Let's run a small benchmark to prove this point.

First, let's reference Newtonsoft.Json.

In [1]:
#r "nuget:Newtonsoft.Json"

Installed package Newtonsoft.Json version 12.0.3

Next, we'll define a small benchmark suite utility using `Nuqleon.Time`'s support to build custom stopwatches to measure bytes allocated.

In [1]:
using System.Time;

class MemoryClock : IClock
{
    public long Now => GC.GetAllocatedBytesForCurrentThread();
}

IStopwatch swMem = StopwatchFactory.FromClock(new MemoryClock()).Create();
IStopwatch swTime = StopwatchFactory.Diagnostics.Create();

void Benchmark(string title, Action test, int n)
{
    swMem.Restart();
    swTime.Restart();

    for (int i = 0; i < n; i++)
    {
        test();
    }

    swTime.Stop();
    swMem.Stop();

    Console.WriteLine($"{title} completed in {swTime.ElapsedMilliseconds} ms and allocated {swMem.ElapsedTicks} bytes.");
}

Now we can define two serializer functions, one using Newtonsoft and one using Nuqleon. In both cases, we'll use `StringWriter.Null` to emit the JSON, so we just account for the overheads of the library used.

In [1]:
using System.IO;
using System.Text;
using Newtonsoft.Json;

var serN = new Newtonsoft.Json.JsonSerializer();
var serJ = FastJsonSerializerFactory.CreateSerializer<Person>(DefaultNameProvider.Instance, new FastJsonSerializerSettings(FastJsonConcurrencyMode.SingleThreaded));

void TestN() => serN.Serialize(StringWriter.Null, p);
void TestJ() => serJ.Serialize(p, StringWriter.Null);

Finally, let's run the benchmark.

In [1]:
Benchmark("Newtonsoft", TestN, 1_000_000);
Benchmark("Nuqleon", TestJ, 1_000_000);

Newtonsoft completed in 632 ms and allocated 448201408 bytes.


Nuqleon completed in 98 ms and allocated 216 bytes.


Note that Nuqleon does not allocate any bytes. No intermediate data structures are built to facilitate serialization. The genenerated JSON serialization code is specialized for `Person`, traverses the object graph, and writes directly to the text writer.

### Deserialize

To deserialize objects from JSON, we first create an `IFastJsonDeserializer<T>` by using the `FastJsonSerializerFactory.CreateDeserializer<T>(INameResolver, FastJsonSerializerSettings)` method. The two parameters provided are:

* An `INameResolver` to map a key in a JSON object to the property or field to assign the deserialized value to.
* Settings represented as a `FastJsonSerializerSettings` object. This currently only consist of a concurrency mode, but may be extended in the future.

To illustrate deserialization, let's have a look at creating a deserializer first.

In [1]:
IFastJsonDeserializer<Person> deserializer = FastJsonSerializerFactory.CreateDeserializer<Person>(DefaultNameResolver.Instance, new FastJsonSerializerSettings(FastJsonConcurrencyMode.SingleThreaded));

In this configuration, we use the default name resolver which will use the `Name` and `Age` properties while constructing a `Person` instance. The serializer settings specify a `SingleThreaded` concurrency mode, which makes the serializer only safe to run from a single thread. This enables higher throughput compared to a concurrency-safe instance of a deserializer.

Next, we can use the deserializer to deserialize a `Person` instance.

In [1]:
var json = "{\"Name\": \"Bart\", \"Age\": 21, \"Hobbies\": [ \"Code\", \"Walk\", \"Run\" ]}";

Person p = deserializer.Deserialize(json);

Console.WriteLine(p);

Person { Name = Bart, Age = 21 }


Note that the JSON string we deserialize from contains an additional `Hobbies` property which gets dropped on the floor. This is used to illustrate the strenght of the Nuqleon serializer in terms of efficiency. Because the deserializer is built at runtime from reflecting on the structure of the `Person` type, it can efficiently skip over the whole JSON array and avoid allocating tokens, objects, or even .NET arrays to represent the value that's not needed to construct the `Person` object.

Just like we did earlier for the serializer, let's define a little benchmark to compare the performance of deserialization across different JSON libraries.

In [1]:
var serN = new Newtonsoft.Json.JsonSerializer();
var serJ = FastJsonSerializerFactory.CreateDeserializer<Person>(DefaultNameResolver.Instance, new FastJsonSerializerSettings(FastJsonConcurrencyMode.SingleThreaded));

void TestN() => serN.Deserialize(new StringReader(json), typeof(Person));
void TestJ() => serJ.Deserialize(json);

Finally, let's run the benchmark.

In [1]:
Benchmark("Newtonsoft", TestN, 1_000_000);
Benchmark("Nuqleon", TestJ, 1_000_000);

Newtonsoft completed in 1780 ms and allocated 2736021672 bytes.


Nuqleon completed in 931 ms and allocated 64000048 bytes.


Again, note how the Nuqleon library avoids a lot of unnecessary allocations. There's also no need to allocate an intermediate `TextReader` to deserialize from a string.